In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../Dataset/Revised_Cleaned_Dataset.csv')

In [3]:
df.head()

,eng,Subject,final_text,topic,difficulty
0,An anti-forest measure is\nA. Afforestation\nB...,Biology,antiforest measure afforestation b selective g...,Basic Conceptual Questions,Medium
1,"Among the following organic acids, the acid pr...",Chemistry,among following organic acid acid present ranc...,Acids Bases Salts,Hard
2,If the area of two similar triangles are equal...,Maths,area two similar triangle equal equilateral b ...,Basic Conceptual Questions,Medium
3,"In recent year, there has been a growing\nconc...",Biology,recent year growing concern gradually increasi...,Basic Conceptual Questions,Medium
4,Which of the following statement\nregarding tr...,Physics,following statement regarding transformer inco...,Work Power Energy,Hard


In [4]:
df = df.dropna(subset=['final_text'])


In [5]:
df.isnull().sum()

eng           0
Subject       0
final_text    0
topic         0
difficulty    0
dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

topic_encoder = LabelEncoder()
difficulty_encoder = LabelEncoder()

In [7]:
df['topic'] = topic_encoder.fit_transform(df['topic'])
df['difficulty'] = difficulty_encoder.fit_transform(df['difficulty'])

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1,2)
)

In [9]:
X = tfidf.fit_transform(df['final_text'])

In [10]:
from sklearn.model_selection import train_test_split

y_topic = df['topic']

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X, y_topic, test_size=0.2, random_state=42, stratify=y_topic
)


In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

models = {
    "NB": MultinomialNB(),
    "LR": LogisticRegression(max_iter=1000),
    "SVM": LinearSVC()
}

for name, model in models.items():
    model.fit(X_train_t, y_train_t)
    acc = model.score(X_test_t, y_test_t)
    print(name, acc)


NB 0.6623323519790645
LR 0.8281403336604515
SVM 0.862855740922473


We will hyperparameter tune the svm model to improve our accuracy

In [12]:
svm = LinearSVC()

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {
    'C': [0.01, 0.1, 1, 5, 10],
    'loss': ['hinge', 'squared_hinge']
}

In [15]:
svm = LinearSVC()
grid = GridSearchCV(
    svm,
    param_grid,
    scoring='accuracy',
    cv=3,          
    n_jobs=-1
)

In [16]:
grid.fit(X_train_t, y_train_t)

c:\Users\Aaru\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LinearSVC()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.01, 0.1, ...], 'loss': ['hinge', 'squared_hinge']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold an

In [17]:
print("Best Params:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

Best Params: {'C': 5, 'loss': 'hinge'}
Best CV Accuracy: 0.8552575101776844


In [18]:
best_svc = grid.best_estimator_

from sklearn.metrics import accuracy_score

test_acc = accuracy_score(y_test_t, best_svc.predict(X_test_t))
print("Test Accuracy:", test_acc)


Test Accuracy: 0.8659633627739614


In [19]:
y_difficulty = df['difficulty']

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y_difficulty, test_size=0.2, random_state=42)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

models = {
    "NB": MultinomialNB(),
    "LR": LogisticRegression(max_iter=1000),
    "SVM": LinearSVC()
}

for name, model in models.items():
    model.fit(X_train_d, y_train_d)
    acc = model.score(X_test_d, y_test_d)
    print(name, acc)

NB 0.6484298331697743
LR 0.7481599607458292
SVM 0.7508177952240759


In [21]:
best_svc_d = LinearSVC()
best_svc_d.fit(X_train_d,y_train_d)
test_acc_d = accuracy_score(y_test_d,best_svc_d.predict(X_test_d))
print(test_acc_d)

0.7508177952240759


In [22]:
import pickle

with open("../models/best_model_topic.pkl",'wb') as f:
    pickle.dump(best_svc,f)

with open("../models/best_model_difficulty.pkl",'wb') as f:
    pickle.dump(best_svc_d,f)

with open("../models/label_encoder_topic.pkl",'wb') as f:
    pickle.dump(topic_encoder,f)

with open("../models/label_encoder_difficulty.pkl",'wb') as f:
    pickle.dump(difficulty_encoder,f)